# Building AEMO's demand forecasting neural network from scratch
- Based on AEMO's [Neural Network Documentation](https://aemo.com.au/-/media/files/pdf/so_fd_01__five_minute_electricity_demand_forecasting_neural_network_documentation.pdf)
- The model predicts the logarithmic change in demand for the region
- The input data, weights, and biases are based on the NSW neural network that is described in the documentation
- The documentation appears to have combined the weights and biases together in the same table
- The neural network is based on this tutorial [video](https://www.youtube.com/playlist?list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3) series and [book](https://nnfs.io/)

In [1]:
import numpy as np

import pandas as pd
from datetime import datetime, timedelta

np.random.seed(0)

In [2]:
nsw_df = pd.read_csv('data/nsw_example_data.csv', parse_dates=['Date'], dayfirst=True)
nsw_df

,Date,Demand
0,1998-02-01 00:00:00,6010
1,1998-02-01 00:05:00,5990
2,1998-02-01 00:10:00,6000
3,1998-02-01 00:15:00,5970
4,1998-02-01 00:20:00,5960
5,1998-02-01 00:25:00,5880
6,1998-02-08 00:00:00,6250
7,1998-02-08 00:05:00,6280
8,1998-02-08 00:10:00,6180
9,1998-02-08 00:15:00,6210


### Inputs
- The neural network has 9 input values based on:
    - The logarithmic changes in demand in the 4 dispatch intervals immediately before the interval the prediction is being run for
    - The 5 dispatch intervals precisely a week before
- AEMO has provided the pretrained weights and biases

In [3]:
def gen_inputs(df):
    '''
    Generate the 9 input values to predict demand for the next 5 minute dispatch interval
    '''
    # the datetime of the next dispatch datetime
    ddt = nsw_df['Date'].max() + timedelta(minutes=5)

    dt11 = df[df['Date']==ddt - timedelta(days=7, minutes=25)]['Demand'].iloc[0]
    dt10 = df[df['Date']==ddt - timedelta(days=7, minutes=20)]['Demand'].iloc[0]
    dt9 = df[df['Date']==ddt - timedelta(days=7, minutes=15)]['Demand'].iloc[0]
    dt8 = df[df['Date']==ddt - timedelta(days=7, minutes=10)]['Demand'].iloc[0]
    dt7 = df[df['Date']==ddt - timedelta(days=7, minutes=5)]['Demand'].iloc[0]
    dt6 = df[df['Date']==ddt - timedelta(days=7, minutes=0)]['Demand'].iloc[0]

    dt5 = df[df['Date']==ddt - timedelta(days=0, minutes=25)]['Demand'].iloc[0]
    dt4 = df[df['Date']==ddt - timedelta(days=0, minutes=20)]['Demand'].iloc[0]
    dt3 = df[df['Date']==ddt - timedelta(days=0, minutes=15)]['Demand'].iloc[0]
    dt2 = df[df['Date']==ddt - timedelta(days=0, minutes=10)]['Demand'].iloc[0]
    dt1 = df[df['Date']==ddt - timedelta(days=0, minutes=5)]['Demand'].iloc[0]

    X = np.array([
        # 1 week before
        np.log(dt10/dt11),
        np.log(dt9/dt10),
        np.log(dt8/dt9),
        np.log(dt7/dt8),
        np.log(dt6/dt7),
        # 4 dispatch intervals before
        np.log(dt4/dt5),
        np.log(dt3/dt4),
        np.log(dt2/dt3),
        np.log(dt1/dt2),
    ])

    return X

X = gen_inputs(nsw_df)
X

array([-0.00333334,  0.00166806, -0.00501254, -0.00167645, -0.01351372,
        0.00478852, -0.01605171,  0.00484262, -0.00808412])

In [4]:
layer_1_weights = np.array([
    [0.787908442,-3.03342919,-0.805006387,-2.24481232,-6.91548304,1.899275,1.67724099,3.34159312,2.33262311],
    [-0.280762392,-1.28836905,-1.64200928,-2.93899286,-0.413204144,2.10931932,-0.0174202002,-0.498683481,1.10089596],
    [-0.0541686846,-0.00341524871,0.662373364,0.409496988,2.02470863,-0.140064819,0.0654530737,0.0654530737,-1.07629121],
    [-0.07762109,-0.161543795,0.344925654,1.99314546,0.843839487,0.648678667,0.752352854,1.15456333,0.839209192],
]).T

output_weights = np.array([0.171686888,-0.134112006, 1.06132145,1.9234954])

layer_1_biases = np.array([-1.18083652,0.912479873,0.168973233,-1.92511602])
output_biases = np.array([-0.766221613])

## Neural network implementation

In [5]:
class Layer_Dense:
    def __init__(self, n_inputs=None, n_neurons=None, weights=None, biases=None):
        if weights is None:
            self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
            self.biases = np.zeros((1, n_neurons))
        else:
            self.weights = weights
            self.biases = biases
    
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + np.exp(-inputs))

## Define the model
- 9 neurons in the input layer
- 4 neurons in the hidden layer
- 1 output neuron
- Add the pretrained weights/biases

In [6]:
layer_1 = Layer_Dense(9, 4, layer_1_weights, layer_1_biases)
activation_1 = Activation_Sigmoid()

output_layer = Layer_Dense(4, 1, output_weights, output_biases)
activation_output = Activation_Sigmoid()

## Neural network forward propagation

In [7]:
layer_1.forward(X)
activation_1.forward(layer_1.output)

output_layer.forward(activation_1.output)
activation_output.forward(output_layer.output)

## Prediction
- To get the predicted logarithmic change, multiply the output by 2 and minus 1

In [10]:
2 * activation_output.output - 1

array([-0.00572127])